**File Description**

I made this file later on when I tried to revisit the ML process for the nfl. I created a new file for feature engineering based on the work that I had 
done on qb passing yards. I was thinking I should have a file that engineers the features and the files with the models simply read from it. I did not
get to work with these features much but it gave me a starting point on how I might set up the procecss for the NFL.

**AI-Usage** 

I utilized chat gpt to help me throughout this process. It gave me an outline of how to extract the features and target variables, intialize the models, and compute performance metrics to analyze the different models. After understanding how to perform this process for one model, I got help creating loops to loop through the different models and print out the performance metrics in an organized manner. Upon completing this portion of the machine learning process, it helped guide me towards understanding which features are most important and how to make predictions for a specfic player's passing yards. I provided more details towards how I engineered the code using prompts above their code blocks. 

The code developed in this file was guided by this conversation with ChatGPT.

**Here is the link to that chat:** https://chatgpt.com/share/67a7d9ef-d054-800f-a70d-36bf42bdf994

In [6]:
import pandas as pd
import numpy as np

# Load the dataset
qb_df = pd.read_csv("../clean_qb.csv")

display(qb_df)

# Checking to see if the list contains only quarterbacks 
print(qb_df['pfr_player_name'].unique()) 

,player_id,player_name,pfr_player_name,position,recent_team,week,season_type,opponent_team,passing_yards,passing_tds,...,interceptions,passer_rating,completions,completion_percentage,expected_completion_percentage,completion_percentage_above_expectation,avg_air_distance,max_air_distance,offense_snaps,offense_pct
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,1,REG,SF,167,1,...,1.0,82.837302,13.0,61.904762,72.558512,-10.653750,19.915377,40.823210,38,0.75
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,2,REG,TEN,176,2,...,0.0,98.750000,18.0,60.000000,67.851451,-7.851451,19.087849,52.851567,57,1.00
2,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,3,REG,NE,281,2,...,0.0,118.869048,27.0,77.142857,67.638107,9.504750,19.648404,32.873395,73,1.00
3,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,4,REG,DEN,225,0,...,0.0,72.023810,24.0,57.142857,67.652290,-10.509432,19.960963,39.107162,74,1.00
4,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,5,REG,MIN,244,2,...,3.0,54.861111,29.0,53.703704,66.891636,-13.187932,20.394048,40.734126,72,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,00-0039918,C.Williams,Caleb Williams,QB,CHI,14,REG,SF,134,2,...,0.0,116.938406,17.0,73.913043,73.322051,0.590993,20.952260,36.292903,49,0.92
521,00-0039918,C.Williams,Caleb Williams,QB,CHI,15,REG,MIN,191,1,...,0.0,86.895161,18.0,58.064516,62.604289,-4.539772,22.596859,46.472393,64,0.96
522,00-0039918,C.Williams,Caleb Williams,QB,CHI,16,REG,DET,334,2,...,0.0,107.708333,26.0,65.000000,61.272584,3.727416,21.701507,53.100679,64,1.00
523,00-0039918,C.Williams,Caleb Williams,QB,CHI,17,REG,SEA,122,0,...,1.0,52.976190,16.0,57.142857,63.191599,-6.048742,20.304159,39.255328,61,1.00


['Aaron Rodgers' 'Matthew Stafford' 'Andy Dalton' 'Russell Wilson'
 'Kirk Cousins' 'Geno Smith' 'Derek Carr' 'Jameis Winston' 'Dak Prescott'
 'Jared Goff' 'Jacoby Brissett' 'Deshaun Watson' 'Cooper Rush'
 'Patrick Mahomes' 'Lamar Jackson' 'Baker Mayfield' 'Josh Allen'
 'Sam Darnold' 'Kyler Murray' 'Gardner Minshew' 'Drew Lock' 'Daniel Jones'
 'Tua Tagovailoa' 'Jordan Love' 'Justin Herbert' 'Jalen Hurts'
 'Joe Burrow' 'Justin Fields' 'Trevor Lawrence' 'Mac Jones' 'Brock Purdy'
 'Bryce Young' 'Will Levis' 'C.J. Stroud' 'Anthony Richardson' 'Bo Nix'
 'Drake Maye' 'Jayden Daniels' 'Caleb Williams']


In [ ]:
# Drop unneccessary columnms
columns_to_drop = [
    'carries', 
    'rushing_yards_before_contact', 
    'rushing_yards_before_contact_avg', 
    'rushing_yards_after_contact', 
    'rushing_yards_after_contact_avg', 
    'rushing_broken_tackles',
    'pass_yards',
    'pass_touchdowns'
]

qb_df = qb_df.drop(columns=columns_to_drop)


In [8]:
# Drop rows with NaN values in any column
qb_df = qb_df.dropna()

# qb_df = pd.get_dummies(qb_df, columns=['recent_team', 'opponent_team'], drop_first=True)

# X = qb_df.drop(columns=['player_id', 'player_name'])

print(qb_df.columns)

Index(['player_id', 'player_name', 'pfr_player_name', 'position',
       'recent_team', 'week', 'season_type', 'opponent_team', 'passing_yards',
       'passing_tds', 'sacks', 'sack_yards', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'rushing_yards', 'rushing_tds',
       'fantasy_points', 'fantasy_points_ppr', 'passing_drops',
       'passing_drop_pct', 'passing_bad_throws', 'passing_bad_throw_pct',
       'times_sacked', 'times_blitzed', 'times_hurried', 'times_hit',
       'times_pressured', 'times_pressured_pct', 'avg_time_to_throw',
       'avg_completed_air_yards', 'avg_intended_air_yards',
       'avg_air_yards_differential', 'aggressiveness',
       'max_completed_air_distance', 'avg_air_yards_to_sticks', 'attempts',
       'interceptions', 'passer_rating', 'completions',
       'completion_percentage', 'expected_completion_percentage',
       'completion_percentage_above_expectation', 

In [ ]:
# Create a copy and sort it by week
qb_df = qb_df.copy()

qb_df = qb_df.sort_values(by=['player_id', 'week'])

# Rolling Averages (Last 3 Games, Last 5 Games, Season Avg)
rolling_windows = [1, 3, 5]  # Last 3 games, last 5 games

rolling_cols = [
    'passing_yards', 'passing_tds', 'sacks', 'sack_yards', 'passing_air_yards',
    'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
    'passing_2pt_conversions', 'pacr', 'rushing_yards', 'rushing_tds',
    'fantasy_points', 'fantasy_points_ppr', 'passing_drops',
    'passing_drop_pct', 'passing_bad_throws', 'passing_bad_throw_pct',
    'times_sacked', 'times_blitzed', 'times_hurried', 'times_hit',
    'times_pressured', 'times_pressured_pct', 'avg_time_to_throw',
    'avg_completed_air_yards', 'avg_intended_air_yards',
    'avg_air_yards_differential', 'aggressiveness',
    'max_completed_air_distance', 'avg_air_yards_to_sticks', 'attempts',
    'interceptions', 'passer_rating',
    'completions', 'completion_percentage',
    'expected_completion_percentage',
    'completion_percentage_above_expectation', 'avg_air_distance',
    'max_air_distance', 'offense_snaps', 'offense_pct'
]

"""
I referred to these conversations when fixing the rolling data frame calculations. I was running into trouble
with my original code as it was not taking into account seperate players correctly and ensuring that each players 
first week of rolling averages was all 0s. I asked for help to fix these aspects of the rolling average calculations 
both the rolling 3 and 5 games along with the season average. It took some back and forth with ChatGPT, however
I was eventaully able to implement these features correctly based on the code provided by using loops for setting 
the initial rows to 0s and using a lambda function to calulcate rolling averages for each player.

I switched chats to ask a question about excel and accidentally continued to use that chat. I included links to all 
conversations utilized to implement this code 

https://chatgpt.com/share/67c7a70a-4c10-800f-acbb-4f83485888e0 
https://chatgpt.com/share/67c7a740-aecc-800f-9566-8ffc39e10d21 
"""

qb_df = qb_df.sort_values(by=['player_id', 'week']).reset_index(drop=True)

# Identify the first week for each player
first_week_indices = qb_df.groupby('player_id').head(1).index

# Initialize rolling average columns with 0 for the first week of each player
for window in rolling_windows:
    for col in rolling_cols:
        qb_df[f'{col}_rolling_{window}'] = 0  # Default all to 0

# Set rolling and seasonal average values to 0 for each QB's first week
qb_df.loc[first_week_indices, [f'{col}_rolling_{window}' for col in rolling_cols for window in rolling_windows]] = 0
qb_df.loc[first_week_indices, [f'{col}_season_avg' for col in rolling_cols]] = 0

# Compute rolling averages for each player separately, using `group_keys=False`
for window in rolling_windows:
    for col in rolling_cols:
        qb_df[f'{col}_rolling_{window}'] = (
            qb_df.groupby('player_id', group_keys=False)[col]
            .apply(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
        )

# Now calculate expanding seasonal averages for each player in a single operation
for col in rolling_cols:
    qb_df[f'{col}_season_avg'] = (
        qb_df.groupby('player_id', group_keys=False)[col]
        .apply(lambda x: x.shift(1).expanding(min_periods=1).mean())
    )

# Reset index to maintain order
qb_df = qb_df.reset_index(drop=True)

# Display the DataFrame
display(qb_df.head())

/tmp/ipykernel_27290/2173590374.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  qb_df[f'{col}_rolling_{window}'] = 0  # Default all to 0
/tmp/ipykernel_27290/2173590374.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  qb_df[f'{col}_rolling_{window}'] = 0  # Default all to 0
/tmp/ipykernel_27290/2173590374.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) ins

,player_id,player_name,pfr_player_name,position,recent_team,week,season_type,opponent_team,passing_yards,passing_tds,...,interceptions_season_avg,passer_rating_season_avg,completions_season_avg,completion_percentage_season_avg,expected_completion_percentage_season_avg,completion_percentage_above_expectation_season_avg,avg_air_distance_season_avg,max_air_distance_season_avg,offense_snaps_season_avg,offense_pct_season_avg
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,1,REG,SF,167,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,2,REG,TEN,176,2,...,1.000000,82.837302,13.000000,61.904762,72.558512,-10.653750,19.915377,40.823210,38.0,0.750000
2,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,3,REG,NE,281,2,...,0.500000,90.793651,15.500000,60.952381,70.204982,-9.252601,19.501613,46.837388,47.5,0.875000
3,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,4,REG,DEN,225,0,...,0.333333,100.152116,19.333333,66.349206,69.349357,-3.000151,19.550543,42.182724,56.0,0.916667
4,00-0023459,A.Rodgers,Aaron Rodgers,QB,NYJ,5,REG,MIN,244,2,...,0.250000,93.120040,20.500000,64.047619,68.925090,-4.877471,19.653148,41.413833,60.5,0.937500


In [10]:
# Replace all NaN values with 0 in the entire DataFrame
qb_df = qb_df.fillna(0)

# create a csv file to store the data
qb_df.to_csv("../rolling_qb_features.csv", index=False)